In [1]:
import numpy as np
import pandas as pd
from tqdm import trange
from tqdm import tqdm
import time

In [2]:
data=pd.read_csv("group_call_HMS.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6962341 entries, 0 to 6962340
Data columns (total 5 columns):
call_start_day     int64
call_start_time    object
call_end_day       int64
call_end_time      object
hashed             object
dtypes: int64(2), object(3)
memory usage: 265.6+ MB


In [4]:
len(data)

6962341

### 총 통화 횟수, 총 통화 시간, 평균 통화 시간, 총 통화한 인원수, 1회 통화에 참여한 평균 인원수

In [5]:
# 시간을 초로 계산 (연산 편하게 하기 위해 ) 시*3600,분 60, 초
def time_sec(timestr):
    sec=int(timestr[:2])*3600+int(timestr[3:5])*60+int(timestr[6:8])
    return sec


## - 통화시간 탐색할 generator

In [6]:
time_gap_sec=(time_sec(end)-time_sec(start) if (end_day-start_day)==0
             else (end_day-start_day)*3600*24-(time_sec(end)-time_sec(start))
             for end, start, end_day, start_day in zip(data.call_end_time, data.call_start_time,
                                                      data.call_end_day,data.call_start_day))

In [7]:
time_gap_sec

<generator object <genexpr> at 0x0000019A64E92EB8>

In [8]:
time_gap_list=[]
for i in trange(len(data)):
    iterator_time=next(time_gap_sec)
    time_gap_list.append(iterator_time)
    

100%|████████████████████████████████████████████████████████████████████| 6962341/6962341 [00:44<00:00, 157292.42it/s]


In [9]:
# generator 생성
hash_list=(i for i in data['hashed'])
print(hash_list)

<generator object <genexpr> at 0x0000019A000596D0>


In [10]:
#통화한 사람 수 => hash_list의 ,으로 구분된 길이
num_people_list=[]
for i in trange(len(data)):
    iterator_nump=next(hash_list)
    #print(iterator_nump)
    num=len(iterator_nump.split(','))
    num_people_list.append(num)
    

100%|████████████████████████████████████████████████████████████████████| 6962341/6962341 [00:19<00:00, 355192.56it/s]


In [11]:
len(num_people_list)

6962341

In [12]:
# 한개의 call 당 통화한 사람 수 
data['num_people']=num_people_list
#한개의 call당 통화시간 
data['time_gap']=time_gap_list
# 한개의 call당 평균 통화시간 
data['each_call_mean']=(np.array(time_gap_list)/np.array(num_people_list))

In [13]:
#한개의 call당 통화수, 통화시간, 한사람당 평균 통화시간
data[:2]

,call_start_day,call_start_time,call_end_day,call_end_time,hashed,num_people,time_gap,each_call_mean
0,1,09:14:58.558,1,09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,3,1592,530.666667
1,17,11:05:05.176,17,13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,5,7357,1471.400000


In [14]:
new_data=data[['hashed','num_people','time_gap','each_call_mean']]

In [15]:
len(str(new_data['hashed'][0]).split(','))

3

# 개개인의 data를 뽑아내기 위해 data 에서 hashed를 개개인으로 해체한다!

In [16]:
# 각 칼럼의 generator 생성
new_hash_list=(i for i in new_data.hashed)
new_num_people=(i for i in new_data.num_people)
new_time_gap=(i for i in new_data.time_gap)

In [17]:
name_list=[]
num_people=[]
time_gap=[]

In [18]:

for i in trange(len(new_data)):
    iterator_hash=next(new_hash_list)
    iterator_npeople=next(new_num_people)
    iterator_tgap=next(new_time_gap)
    # 한사람씩 접근하는 iterator 생성
    iterator_persons=(i for i in iterator_hash.split(','))
    for j in iterator_persons:
        #print(j,'\n')
        # 같은 hashed에 있으면 (한 call)당 나머지 칼럼의 내용은 같게 한다. 
        name_list.append(j)
        num_people.append(iterator_npeople)
        time_gap.append(iterator_tgap)
    
    

100%|████████████████████████████████████████████████████████████████████| 6962341/6962341 [01:04<00:00, 107875.65it/s]


In [19]:
each_data=pd.DataFrame({'name_list':name_list,'num_people':num_people,'time_gap':time_gap})

In [20]:
each_data['num_people']=each_data['num_people'].astype(int)
each_data['time_gap']=each_data['time_gap'].astype(float)

In [21]:
#name_list기준 num_people합계
#총 통화 인원수가 될지어다
df=each_data.groupby('name_list')['num_people'].sum().to_frame()
df=df.join(each_data['name_list'].value_counts())
# 총 통화시간이 될지어다
df=df.join(each_data.groupby('name_list')['time_gap'].sum())

In [22]:
df.columns=['count_call_people','count_call','sum_time']


In [23]:
df['mean_time']=df['sum_time']/df['count_call']

df['mean_people']=df['count_call_people']/df['count_call']

df['mean_people']=df['mean_people'].astype(int)



In [24]:
df[:4]

,count_call_people,count_call,sum_time,mean_time,mean_people
name_list,,,,,
000020b4fa2af1e40c813436e5054bce70b703d0039dcdad4ccb5b287c3fadbc,6370,1049,6317819.0,6022.706387,6
0000264b01392acfde44f9d8494f112a701dc5d3e5fda6ad090cb205727637e0,3625,777,4308817.0,5545.453024,4
00006916672106f94f68d18e4a0532b53a40f4549d3a62f0cbf677dd25803668,48,9,4183.0,464.777778,5
0000988c5778e9d6a861fee8fbfd88ec2fdc9108c1acd6d9c0b4d51637787b37,224,37,193470.0,5228.918919,6
